In [1]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('"Top 10 Funds by AUM Positioning - Data Gen.ipynb"')))))
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('"Top 10 Funds by AUM Positioning - Data Gen.ipynb"')))))
import apis.pa.mypaengine as pa
import apis.qe.myqengine as qe

Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 5


In [2]:
time_series = qe.TimeSeries(start_date = '0M',end_date = '0M')

univ = qe.ScreenUniverse(universe_expr = '(ISAV(FFD_SHR_CLS_AUM(0,ACT,ROLL,USD))=1 AND ISAV(P_PRICE(0,USD))=1 AND PROPER(FFD_CLASS_CAT(TEXT))= PROPER("Size and Style") AND UPPERCASE(P_EXCOUNTRY(ISO2))=UPPERCASE("US") AND PROPER(FFD_CLASS_ASSET(TEXT))=PROPER("EQUITY") AND FFD_ACTM = 1 AND IF(UPPERCASE(FFD_BRAND)=UPPERCASE("DFA"),0,1)=1)=1')

req = qe.calculate(universe = univ, dates = time_series,formulas = ["FFD_NAME",'P_PRICE(0,USD)',"FFD_SHR_CLS_AUM(0,ACT,ROLL,USD)","LSS_TICKER","FFD_BMK_ID","FFD_BRAND"])

In [3]:
df = req.data.copy()
df = df.rename(columns = {"FFD_SHR_CLS_AUM(0,ACT,ROLL,USD)":'aum',
                       "LSS_TICKER":'lionshares_id',
                       "FFD_BMK_ID":'bench_id'})
df_append= pd.DataFrame()
bench_list = ['SP50','R.1000','R.1000G','R.1000V','R.2000','R.2000V','R.2000G','R.3000','990300','990100','899901','891800']
for bench in bench_list:
    if (bench=='SP50'):
        count = 25
    else:
        count =10
    df_append = pd.concat([df_append,df[df['bench_id']==bench].dropna().sort_values('aum',ascending=False).head(count)])

df_append['pa_port_id'] = "LION:"+df_append['lionshares_id']
df_append['pa_bench_id'] = "BENCH:"+df_append['bench_id']

df_append

,DATE,UNIVERSE,FFD_NAME,"P_PRICE(0,USD)",aum,lionshares_id,bench_id,FFD_BRAND,pa_port_id,pa_bench_id
3432,20230831,JUEMX,JPMorgan Trust I,20.36,1.376883e+10,JUEMX-US,SP50,JPMorgan,LION:JUEMX-US,BENCH:SP50
3414,20230831,JHEQX,JPMorgan Trust I,28.17,1.333799e+10,JHEQX-US,SP50,JPMorgan,LION:JHEQX-US,BENCH:SP50
4676,20230831,HGIFX,Hartford Mutual Funds Inc,44.37,9.628773e+09,HGIFX-US,SP50,Hartford,LION:HGIFX-US,BENCH:SP50
2274,20230831,POAGX,Primecap Odyssey Funds,43.21,7.213679e+09,POAGX-US,SP50,PrimeCap,LION:POAGX-US,BENCH:SP50
2085,20230831,JDEUX,JPMorgan Trust I,34.93,6.529215e+09,JDEUX-US,SP50,JPMorgan,LION:JDEUX-US,BENCH:SP50
...,...,...,...,...,...,...,...,...,...,...
3813,20230831,POEFX,Principal Funds Inc,9.46,2.581962e+09,POEFX-US,891800,Principal,LION:POEFX-US,BENCH:891800
760,20230831,LZEMX,Lazard Funds Inc,16.44,2.372389e+09,LZEMX-US,891800,Lazard,LION:LZEMX-US,BENCH:891800
2794,20230831,BGKEX,Baillie Gifford Funds,18.82,2.185520e+09,BGKEX-US,891800,Baillie Gifford Funds,LION:BGKEX-US,BENCH:891800
561,20230831,GEMIX,Goldman Sachs Emerging Markets Fund,21.05,2.038520e+09,GEMIX-US,891800,Goldman Sachs,LION:GEMIX-US,BENCH:891800


In [4]:
df_append.to_csv('../../data/Top Fund Positioning/fund_screen_20230831.csv')

In [5]:
pa_document_name = "PERSONAL:API_SP"
pa_component_category = "main / Weights"
pa_component_name = "Weights"

#Pass our to PA inputs to our custom class
pa_doc = pa.DocumentTemplate(
                    pa_document_name = pa_document_name,
                    pa_component_category = pa_component_category,
                    pa_component_name = pa_component_name,
                    )


In [6]:
#Define PA Inputs
temp_df =df_append[df_append['bench_id']=='SP50']
portfolios = temp_df['pa_port_id'].to_list()
benchmarks = temp_df['pa_bench_id'].to_list()
enddate = '0M'
pa_doc.run_calc(
               portfolios,
               benchmarks,
               end_date = enddate,
               curr = 'USD',
               mode= 'B&H'
              )
df_full = pd.DataFrame()
for x in list(pa_doc.data.keys()):
    temp = pa_doc.data[x][0]
    temp['port_x_bench'] = x
    df_full = pd.concat([df_full,temp])

In [7]:
pa_document_name = "PERSONAL:API_RUSSELL"
pa_component_category = "main / Weights"
pa_component_name = "Weights"

#Pass our to PA inputs to our custom class
pa_doc = pa.DocumentTemplate(
                    pa_document_name = pa_document_name,
                    pa_component_category = pa_component_category,
                    pa_component_name = pa_component_name,
                    )
portfolios = []
benchmarks = []

#set Russel benchmarks
rus_benchs = ['R.1000','R.1000G','R.1000V','R.2000','R.2000V','R.2000G','R.3000']

#Define PA Inputs
temp_df = df_append[df_append['bench_id'].isin(rus_benchs)]
enddate = '0M'
#Run PA Calc
pa_doc.run_calc(
            temp_df['pa_port_id'].to_list(),
            temp_df['pa_bench_id'].to_list(),
            end_date = enddate,
            curr = 'USD',
            mode= 'B&H'
            )
#Append to dataframe
for x in list(pa_doc.data.keys()):
    temp = pa_doc.data[x][0]
    temp['port_x_bench'] = x
    df_full = pd.concat([df_full,temp])
        

In [8]:
pa_document_name = "PERSONAL:API_MSCI"
pa_component_category = "main / Weights"
pa_component_name = "Weights"

#Pass our to PA inputs to our custom class
pa_doc = pa.DocumentTemplate(
                    pa_document_name = pa_document_name,
                    pa_component_category = pa_component_category,
                    pa_component_name = pa_component_name,
                    )
msci_benchs = ['990300','990100','899901','891800']

temp_df = df_append[df_append['bench_id'].isin(msci_benchs)]
enddate = '0M'
pa_doc.run_calc(
            temp_df['pa_port_id'].to_list(),
            temp_df['pa_bench_id'].to_list(),
            end_date = enddate,
            curr = 'USD',
            mode= 'B&H'
            )
for x in list(pa_doc.data.keys()):
    temp = pa_doc.data[x][0]
    temp['port_x_bench'] = x
    df_full = pd.concat([df_full,temp])

In [9]:
#Cleaning up data
df_full=df_full.replace('',np.nan)
df_full["GICS Sector"]= df_full["GICS Sector"].replace('--','Other')
df_full["GICS Sub Industry"]= df_full["GICS Sub Industry"].replace('--','Other')
df_full["GICS Industry Name"]= df_full["GICS Industry Name"].replace('--','Other')
df_full["GICS Industry Group"]= df_full["GICS Industry Group"].replace('--','Other')
df_full["Asset Type+(Client Definition/+FactSet)"]= df_full["Asset Type+(Client Definition/+FactSet)"].replace('--','Other')
df_full["Asset Class+(Client Definition/+FactSet)"]= df_full["Asset Class+(Client Definition/+FactSet)"].replace('--','Other')
df_full[['Port.+Weight','Bench.+Weight','Difference']] = df_full[['Port.+Weight','Bench.+Weight','Difference']].fillna(0)
df_full[['Port.+Weight','Bench.+Weight','Difference']] =df_full[['Port.+Weight','Bench.+Weight','Difference']].astype(float)

pattern = r'LION:(.*?)_x_'
df_full['lionshares_id'] = df_full['port_x_bench'].str.extract(pattern, expand=False)

#df_full=df_full.drop(columns = 'portfolio')

df_full= df_full.set_index(['lionshares_id','Symbol']).join(df.set_index('lionshares_id'),how='left',on='lionshares_id')

In [10]:

df_full = df_full.rename(columns ={'Asset Type+(Client Definition/+FactSet)':'Asset Type',
                        'Asset Class+(Client Definition/+FactSet)':'Asset Class',
                        'price':'Fund Price',
                        'security0':'Security Name',
                        'Port.+Weight':'Portfolio Weight',
                        'Bench.+Weight':'Benchmark Weight',
                        'Difference':'Active Weight',
                        'UNIVERSE':'Fund Ticker',
                        'FFD_BRAND':'Fund Brand'
                        })

df_full = df_full.drop(columns = ['port_x_bench'])

In [11]:
df_full.to_pickle('../../data/Top Fund Positioning/t10_full_universe_weights_20230831.pkl')